<a href="https://colab.research.google.com/github/josharsh/100LinesOfCode/blob/master/Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow==1.15.0

In [0]:
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()

In [0]:

users = ['Ryan', 'Danielle',  'Vijay', 'Chris']
movies = ['Star Wars', 'The Dark Knight', 'Shrek', 'The Incredibles', 'Bleu', 'Memento']
features = ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
num_users = len(users)
num_movies = len(movies)
num_feats = len(features)
num_recommendations = 2

In [0]:
users_movies = tf.constant([
                [4,  6,  8,  0, 0, 0],
                [0,  0, 10,  0, 8, 3],
                [0,  6,  0,  0, 3, 7],
                [10, 9,  0,  5, 0, 2]],dtype=tf.float32)

In [0]:
movies_feats = tf.constant([
                [1, 1, 0, 0, 1],
                [1, 1, 0, 0, 0],
                [0, 0, 1, 1, 0],
                [1, 0, 1, 1, 0],
                [0, 0, 0, 0, 1],
                [1, 0, 0, 0, 1]],dtype=tf.float32)

In [0]:
users_feats=tf.matmul(users_movies,movies_feats)


In [17]:
users_feats = users_feats/tf.reduce_sum(users_feats,axis=1,keepdims=True)
users_feats

<tf.Tensor: id=114, shape=(4, 5), dtype=float32, numpy=
array([[0.25      , 0.25      , 0.2       , 0.2       , 0.1       ],
       [0.0882353 , 0.        , 0.29411766, 0.29411766, 0.32352942],
       [0.44827586, 0.20689656, 0.        , 0.        , 0.3448276 ],
       [0.3880597 , 0.2835821 , 0.07462686, 0.07462686, 0.17910448]],
      dtype=float32)>

In [18]:
top_users_features = tf.nn.top_k(users_feats, num_feats)[1]
top_users_features

<tf.Tensor: id=117, shape=(4, 5), dtype=int32, numpy=
array([[0, 1, 2, 3, 4],
       [4, 2, 3, 0, 1],
       [0, 4, 1, 2, 3],
       [0, 1, 4, 2, 3]], dtype=int32)>

In [19]:
for i in range(num_users):
    feature_names = [features[index] for index in top_users_features[i]]
    print('{}: {}'.format(users[i],feature_names))

Ryan: ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
Danielle: ['Drama', 'Comedy', 'Cartoon', 'Action', 'Sci-Fi']
Vijay: ['Action', 'Drama', 'Sci-Fi', 'Comedy', 'Cartoon']
Chris: ['Action', 'Sci-Fi', 'Drama', 'Comedy', 'Cartoon']


In [0]:
movies_feats_trans=tf.transpose(movies_feats)
users_feats_trans=tf.transpose(users_feats)

In [23]:
users_ratings =  tf.matmul(movies_feats,users_feats_trans)
users_ratings

<tf.Tensor: id=220, shape=(6, 4), dtype=float32, numpy=
array([[0.6       , 0.4117647 , 1.        , 0.8507463 ],
       [0.5       , 0.0882353 , 0.6551724 , 0.6716418 ],
       [0.4       , 0.5882353 , 0.        , 0.14925373],
       [0.65      , 0.67647064, 0.44827586, 0.53731346],
       [0.1       , 0.32352942, 0.3448276 , 0.17910448],
       [0.35      , 0.4117647 , 0.79310346, 0.5671642 ]], dtype=float32)>

In [0]:
users_movies
users_ratings
user_r=tf.transpose(users_ratings);

In [30]:
users_ratings_new = tf.where(tf.equal(users_movies, tf.zeros_like(users_movies)),
                                  user_r,
                                  tf.zeros_like(tf.cast(users_movies, tf.float32)))
users_ratings_new

<tf.Tensor: id=233, shape=(4, 6), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.65      , 0.1       ,
        0.35      ],
       [0.4117647 , 0.0882353 , 0.        , 0.67647064, 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , 0.44827586, 0.        ,
        0.        ],
       [0.        , 0.        , 0.14925373, 0.        , 0.17910448,
        0.        ]], dtype=float32)>

In [31]:
top_movies = tf.nn.top_k(users_ratings_new, num_recommendations)[1]
top_movies

<tf.Tensor: id=236, shape=(4, 2), dtype=int32, numpy=
array([[3, 5],
       [3, 0],
       [0, 3],
       [4, 2]], dtype=int32)>

In [32]:

for i in range(num_users):
    movie_names = [movies[index] for index in top_movies[i]]
    print('{}: {}'.format(users[i],movie_names))

Ryan: ['The Incredibles', 'Memento']
Danielle: ['The Incredibles', 'Star Wars']
Vijay: ['Star Wars', 'The Incredibles']
Chris: ['Bleu', 'Shrek']
